In [9]:
#install module and load it into notebook
!pip install textblob
!pip install spacy
!pip install gensim
!python -m spacy download en
!python -m spacy download en_core_web_lg
!python -m textblob.download_corpora
import spacy
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
from textblob.sentiments import NaiveBayesAnalyzer
from pprint import pprint
import pandas as pd
import numpy as np
import io
import json
from pandas.io.json import json_normalize
import boto3

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/en_core_web_sm
    -->
    /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/en_core_web_lg
    -->
    /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')

[nltk_data] Downloading package brown to /home/ec2-user/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[n

In [10]:
bucket = 'sentimentdataedgar'
folder = 'data'

# get the region (pay attention to the region of your S3, SageMaker ... keep them in same region where possible)
region = boto3.Session().region_name

In [12]:
# create the s3 client
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key=f'{folder}/yelp_academic_dataset_review_Small.json')
file_content = obj['Body'].read().decode('utf-8') 

#determine data architecture
cols = ['review_id', 'user_id', 'business_id','text', 'polarity','subjectivity', 'sentiment', 'classi']
cols2 = ['text', 'sentiment']
lst = []
lst2 = []

#create variables for DF from json files
print('TextBlob on Action')
for kv in file_content.split("\n"):
    json_content = json.loads(kv)
    review_id = json_content['review_id']
    user_id = json_content['user_id']
    business_id = json_content['business_id']
    sentimet_text = json_content['text']

#create Textblob analysis and variables for polarity and subjectivity
    textBlobResp = TextBlob(sentimet_text)
    polarity = textBlobResp.sentiment.polarity
    subjectivity = textBlobResp.sentiment.subjectivity
    #Use TextBlob NaiveBayer Sentiment Analizer
    blob = TextBlob(sentimet_text, analyzer=NaiveBayesAnalyzer())
    classi = blob.sentiment.classification
    #create variable according to polarity score
    sentiment = np.where(polarity >= 0, 'pos','neg')
    
#build dfs
    lst.append([review_id, user_id, business_id, sentimet_text,polarity, subjectivity, sentiment, classi])
    lst2.append([sentimet_text,sentiment])
print('TextBlob is done')
df1 = pd.DataFrame(lst, columns=cols)
df_train = pd.DataFrame(lst2, columns=cols2)

TextBlob on Action
TextBlob is done


In [13]:
df1

,review_id,user_id,business_id,text,polarity,subjectivity,sentiment,classi
0,x7mDIiDB3jEiPGPHOmDzyw,msQe1u7Z_XuqjGoqhB0J5g,iCQpiavjjPzJ5_3gPD5Ebg,The pizza was okay. Not the best I've had. I p...,0.416667,0.316667,pos,pos
1,dDl8zu1vWPdKGihJrwQbpw,msQe1u7Z_XuqjGoqhB0J5g,pomGBqfbxcqPv14c3XH-ZQ,I love this place! My fiance And I go here atl...,0.550926,0.768519,pos,neg
2,LZp4UX5zK3e-c5ZGSeo3kA,msQe1u7Z_XuqjGoqhB0J5g,jtQARsP6P-LbkyjbO1qNGg,Terrible. Dry corn bread. Rib tips were all fa...,-0.533333,0.800000,neg,neg
3,Er4NBWCmCD4nM8_p1GRdow,msQe1u7Z_XuqjGoqhB0J5g,elqbBhBfElMNSrjFqW3now,Back in 2005-2007 this place was my FAVORITE t...,0.185736,0.462784,pos,neg
4,jsDu6QEJHbwP2Blom1PLCA,msQe1u7Z_XuqjGoqhB0J5g,Ums3gaP2qM3W1XcA5r6SsQ,Delicious healthy food. The steak is amazing. ...,0.546429,0.609524,pos,pos
5,pfavA0hr3nyqO61oupj-lA,msQe1u7Z_XuqjGoqhB0J5g,vgfcTvK81oD4r50NMjU2Ag,This place sucks. The customer service is horr...,-0.190476,0.600000,neg,neg
6,brokEno2n7s4vrwmmUdr9w,msQe1u7Z_XuqjGoqhB0J5g,AxeQEz3-s9_1TyIo-G7UQw,"If you like Thai food, you have to try the ori...",0.450617,0.634877,pos,neg
7,kUZWBVZvhWuC8TWUg5AYyA,msQe1u7Z_XuqjGoqhB0J5g,zdE82PiD6wquvjYLyhOJNA,AMAZING!!!\n\n I was referred here by a friend...,0.348775,0.699755,pos,pos
8,wcqt0III88LEcm19IxFFyA,msQe1u7Z_XuqjGoqhB0J5g,EAwh1OmG6t6p3nRaZOW_AA,Ribs = amazing\n2 hour wait time= not so amazi...,0.276786,0.602381,pos,neg
9,LWUtqzNthMM3vpWZIFBlPw,msQe1u7Z_XuqjGoqhB0J5g,atVh8viqTj-sqDJ35tAYVg,"Food is pretty good, not gonna lie. BUT you ha...",0.136984,0.672103,pos,pos


In [14]:
df_train

,text,sentiment
0,The pizza was okay. Not the best I've had. I p...,pos
1,I love this place! My fiance And I go here atl...,pos
2,Terrible. Dry corn bread. Rib tips were all fa...,neg
3,Back in 2005-2007 this place was my FAVORITE t...,pos
4,Delicious healthy food. The steak is amazing. ...,pos
5,This place sucks. The customer service is horr...,neg
6,"If you like Thai food, you have to try the ori...",pos
7,AMAZING!!!\n\n I was referred here by a friend...,pos
8,Ribs = amazing\n2 hour wait time= not so amazi...,pos
9,"Food is pretty good, not gonna lie. BUT you ha...",pos


In [21]:
#create train set as CSV
df_train.to_csv('train_data.csv')
csvfile = open("train_data2.csv", "r")

In [22]:
#Train model on NaiveBayesClassifier
with csvfile as fp:
    cl = NaiveBayesClassifier(fp, format="csv")

In [23]:
#Measure model accuracy with train set
test_csvfile = open("test_data.csv", "r")
cl.accuracy(test_csvfile)

0.6363636363636364